In [13]:

import requests
import sqlite3
import pandas as pd
from sodapy import Socrata

In [14]:
# api endpoint
api_url = "https://data.colorado.gov/resource/4ykn-tg5h.json"
app_token = "0NT6cCdGg6VVAj2pRLjmAqDFu"
username = "glissmann.mark@gmail.com"
password = "K2PRH@mUDwgbfsV"
entityid = "18881009142"
api_key = "gxc0jsptwdopcqxffpl21c8b"
api_secret_key = "2w9yp2sqyp3d7nxkidj8v65e6ow7mos3e7bnuc64rqaiccyc4c" 


In [15]:
batch_size = 1000 
offset = 0

In [16]:
# connect to SQLite database (or create it)
conn = sqlite3.connect('business_entities.db')
cursor = conn.cursor()

In [17]:
# Fetch one record to determine the structure of the data
response = requests.get(api_url, params={'$limit': 1})
if response.status_code == 200:
    sample_data = response.json()
    if sample_data:
        columns = sample_data[0].keys()
        # Create table based on the keys in the data
        cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS colorado (
            {', '.join([f'{col} TEXT' for col in columns])}
        )
        ''')
        conn.commit()
else:
    print(f"Failed to retrieve data: {response.status_code}")
    exit()

In [18]:
while True:
    # fetch a batch of data
    params = {
        '$limit': batch_size,
        '$offset': offset
    }

    response = requests.get(api_url, params=params)

    if response.status_code == 200:
        data = response.json()

        if not data:
            # no more data to fetch
            break
        
        # prepare insert statement
        placeholders = ', '.join(['?'] * len(columns))
        insert_statement = f'''
        INSERT OR IGNORE INTO colorado ({', '.join(columns)})
        VALUES ({placeholders})
        '''

        # process and insert data into the database
        for item in data:
            values = tuple(item.get(col) for col in columns)
            cursor.execute(insert_statement, values)
        
        # commit changes to the database
        conn.commit()

        # increment offset for the next batch
        offset += batch_size
    else:
        print(f"Failed to retreive data: {response.status_code}")
        break 

# close the database connection
conn.close()